### Part 3 -> Multiturn conversation dataset generation

In [1]:
import json
from tqdm import tqdm
from openai import OpenAI
import random
import pandas as pd
from multiturn_modul import MultiturnStyle, get_multiturn_style

path_to_api_key: str = "../API_KEY.txt"
my_api_key = open(path_to_api_key, 'r').read()
client = OpenAI(api_key=my_api_key)
gpt_model: str = "gpt-4o"

In [43]:
countries = pd.read_csv("../resources/countries_by_income_category.csv")
sampled_country = random.choice(countries.iloc[:, 0].tolist())
id = "0-122"

for i in range(10000):
    test = get_multiturn_style(str(i%12)+"-112", "Switzerland")

multiturn_style_1 = MultiturnStyle(id, sampled_country, nbr_of_turns=2, mode = "normal")

print(multiturn_style_1.system_prompt_chatbot())
print(multiturn_style_1.system_prompt_user())


You are a medical AI chatbot designed to assist health care workers working in Switzerland by answering their questions. 
The style of your answers must follow these rules: 
Provide short and concises answers. 
Provide your answers in flat text. 

You are a physician working in Switzerland using a medical AI chatbot to help you taking care of your patients. 
The initial question that you ask the medical AI chatbot has already been provided. You are now interacting with the medical AI chatbot and asking follow up questions, based on the answers provided by the medical AI chatbot. 
The style of the follow up question you ask must follow these rules: 
Ask short and concises questions. 



In [ ]:
# Test mode
# Generate multiturn conversations between Meditron (simulated by gpt-4o) and a Physician (simulated by gpt-4o)

# System prompts
system_prompt_chatbot = multiturn_style_1.system_prompt_chatbot()  + "In the prompt you see the whole chat history. You are (chatbot). Your task is to answer the questions and only ask follow up questions, if the user did not provide enough information in its input."
system_prompt_user = multiturn_style_1.system_prompt_user()  + "In the prompt you see the whole chat history. You are (user). You want to get an answer to the first question you asked. Your task is to ask followup questions and to interact with the chatbot to get a clear answer to your initial, as a health care worker would do."
system_prompt_chatbot += "\nYour (chatbot) tag is added manually, don't write it in your output. Just generate an answer to the users input"
system_prompt_user += "\nYour (user) tag is added manually, don't write it in your output. Just generate a question or an instruction for the chatbot, based on the conversation history"

# store medical AI tasks in a list
path_to_prompts = "../results/parsed_prompts_tasks_x_subtopics_part2.json" 
with open(path_to_prompts, 'r') as file:
    task_x_subtopics_prompts = json.load(file)  # Load JSON data from file
initial_physician_prompt = task_x_subtopics_prompts[5000]["prompt"]
print(initial_physician_prompt)

# Function that generates the next step of a conversation

def next_conversation_step(system_prompt, chat_histroy, verbose = False):
    if system_prompt == "user":
        content = system_prompt_user
    else:
        content = system_prompt_chatbot

    prompt = chat_histroy

    if verbose:
        print("Send prompt to GPT:")
        print("#########")
        print(prompt)
        print("########")

    completion = client.chat.completions.create(
        model= gpt_model,
        messages=[
            {"role": "system", "content": content},
            {
                "role": "user",
                "content": f"{prompt}"
            }
        ]
    )
    response = completion.choices[0].message.content
    
    if verbose:
        print("Receiving responses from GPT...")
        print(response)
    return chat_histroy + "\n" + f"({system_prompt}) " + response

"How can I explain the basics of BRCA1 and BRCA2 gene mutations and their implications for breast cancer risk in simple terms to help my patient understand their recent genetic test results?"


In [45]:
# Check if everything is fine
N = multiturn_style_1.number_of_turns
print(N)
print(f"chatbot: {system_prompt_chatbot}")
print(f"user: {system_prompt_user}")

2
chatbot: You are a medical AI chatbot designed to assist health care workers working in Switzerland by answering their questions. 
The style of your answers must follow these rules: 
Provide short and concises answers. 
Provide your answers in flat text. 
In the prompt you see the whole chat history. You are (chatbot). Your task is to answer the questions and only ask follow up questions, if the user did not provide enough information in its input.
Your (chatbot) tag is added manually, don't write it in your output. Just generate an answer to the users input
user: You are a physician working in Switzerland using a medical AI chatbot to help you taking care of your patients. 
The initial question that you ask the medical AI chatbot has already been provided. You are now interacting with the medical AI chatbot and asking follow up questions, based on the answers provided by the medical AI chatbot. 
The style of the follow up question you ask must follow these rules: 
Ask short and conc

In [46]:
conversation_history = []
conversation_history.append("(user) " + initial_physician_prompt)
for i in tqdm(range(2*N - 1)): ### Change range -> will cost money
    if i % 2 == 0:
        conversation_history.append(next_conversation_step("chatbot", conversation_history[-1]))
    else:
        conversation_history.append(next_conversation_step("user", conversation_history[-1]))


100%|██████████| 3/3 [00:04<00:00,  1.57s/it]


In [47]:
print(conversation_history[-1])

(user) "How can I explain the basics of BRCA1 and BRCA2 gene mutations and their implications for breast cancer risk in simple terms to help my patient understand their recent genetic test results?"
(chatbot) BRCA1 and BRCA2 are genes that help repair DNA. When they have harmful mutations, this repair process can be impaired, increasing the risk of breast and ovarian cancer. People with these mutations have a higher chance of developing these cancers compared to the general population. Early detection and preventive strategies can help manage this risk.
(user) What preventive strategies can be recommended?
(chatbot) Preventive strategies include regular breast cancer screening like mammograms and MRI, lifestyle changes such as maintaining a healthy weight and limiting alcohol, and medications like tamoxifen or raloxifene for risk reduction. Surgical options, such as prophylactic mastectomy or oophorectomy, may also be considered for high-risk individuals. Discuss personalized recommend